# Load Libraries

In [48]:
# %pip install scikit-learn
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# %pip install tqdm
# %pip install transformers
#%pip install matplotlib

In [49]:
import os
import pandas as pd
import ast
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import BertTokenizer, BertModel
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
import joblib
from tqdm import tqdm
import My_Machine_Learning_Tools as mytools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Load and Explore Data

In [50]:
df_train=pd.read_csv('ModApte_train.csv')
df_test=pd.read_csv('ModApte_test.csv')

In [51]:
def series_to_list(df,column_name):
    result=df[column_name].replace({' ':''},regex=True)
    result.replace({'\\n':''},regex=True,inplace=True)
    result.replace({'\'\'':'\',\''},regex=True,inplace=True)
    return result.apply(ast.literal_eval)

In [52]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9603 entries, 0 to 9602
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         8816 non-null   object
 1   text_type    9603 non-null   object
 2   topics       9603 non-null   object
 3   lewis_split  9603 non-null   object
 4   cgis_split   9603 non-null   object
 5   old_id       9603 non-null   object
 6   new_id       9603 non-null   object
 7   places       9603 non-null   object
 8   people       9603 non-null   object
 9   orgs         9603 non-null   object
 10  exchanges    9603 non-null   object
 11  date         9603 non-null   object
 12  title        9549 non-null   object
dtypes: object(13)
memory usage: 975.4+ KB


In [53]:
df_train.dtypes

text           object
text_type      object
topics         object
lewis_split    object
cgis_split     object
old_id         object
new_id         object
places         object
people         object
orgs           object
exchanges      object
date           object
title          object
dtype: object

In [54]:
df_train.head()

,text,text_type,topics,lewis_split,cgis_split,old_id,new_id,places,people,orgs,exchanges,date,title
0,Showers continued throughout the week in\nthe ...,"""NORM""",['cocoa'],"""TRAIN""","""TRAINING-SET""","""5544""","""1""",['el-salvador' 'usa' 'uruguay'],[],[],[],26-FEB-1987 15:01:01.79,BAHIA COCOA REVIEW
1,The U.S. Agriculture Department\nreported the ...,"""NORM""",['grain' 'wheat' 'corn' 'barley' 'oat' 'sorghum'],"""TRAIN""","""TRAINING-SET""","""5548""","""5""",['usa'],[],[],[],26-FEB-1987 15:10:44.60,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESERVE
2,Argentine grain board figures show\ncrop regis...,"""NORM""",['veg-oil' 'linseed' 'lin-oil' 'soy-oil' 'sun-...,"""TRAIN""","""TRAINING-SET""","""5549""","""6""",['argentina'],[],[],[],26-FEB-1987 15:14:36.41,ARGENTINE 1986/87 GRAIN/OILSEED REGISTRATIONS
3,Moody's Investors Service Inc said it\nlowered...,"""NORM""",[],"""TRAIN""","""TRAINING-SET""","""5551""","""8""",['usa'],[],[],[],26-FEB-1987 15:15:40.12,USX &lt;X> DEBT DOWGRADED BY MOODY'S
4,Champion Products Inc said its\nboard of direc...,"""NORM""",['earn'],"""TRAIN""","""TRAINING-SET""","""5552""","""9""",['usa'],[],[],[],26-FEB-1987 15:17:11.20,CHAMPION PRODUCTS &lt;CH> APPROVES STOCK SPLIT


# Preprocessing

#### Define Parameters

In [55]:
load_encoder=False
fit_encoder=True

#### Define treatment of columns und topics

In [56]:
#define topics
topic_column = 'topics'
food = ['coconut', 'cotton-oil', 'sorghum', 'orange', 'rice', 'soybean', 'sun-meal', 
    'oilseed', 'sugar', 'hog', 'coffee', 'groundnut', 'sunseed', 'sun-oil', 'rye', 
    'lin-oil', 'copra-cake', 'potato', 'barley', 'tea', 'meal-feed', 'coconut-oil', 
    'palmkernel', 'cottonseed', 'castor-oil', 'l-cattle', 'livestock', 'soy-oil', 
    'rape-oil', 'palm-oil', 'cocoa', 'cotton', 'wheat', 'corn', 'f-cattle', 'grain', 
    'soy-meal', 'oat', 'groundnut-oil', 'veg-oil','rapeseed']
resource = ['platinum', 'lead', 'nickel', 'strategic-metal', 'copper', 'palladium', 'gold', 
    'zinc', 'tin', 'iron-steel', 'alum', 'silver', 'nat-gas', 'rubber', 'pet-chem', 'fuel', 'crude','lumber','propane','wool']
finance = ['money-supply', 'dlr', 'nkr', 'lei', 'yen', 'dfl', 'sfr', 'cpi', 'instal-debt', 
    'money-fx', 'gnp', 'interest', 'income', 'dmk', 'rand', 'bop', 'reserves', 'nzdlr','acq']
personal_finance = ['housing','jobs','earn']
transport = ['jet', 'ship']
topics=[[food,'food'],[resource,'resource'],[finance,'finance'],[personal_finance,'personal_finance'],[transport,'transport']]
topics_to_remove = ['gas', 'heat', 'trade', 'retail', 'carcass', 'cpu', 'wpi', 'naphtha', 'ipi','stg','inventories']

#columns with special treatment
list_column='places'
drop_columns=['text_type','people','orgs','exchanges','lewis_split','cgis_split','old_id','new_id']
notnan_columns=['text','topics']
date_columns=['date']
text_columns=['text','title']

#### Define functions for Prepeocessing

These may be turned into a library later.

In [57]:
def drop_row_notnan_columms(df,notnan_columns):
    df_copy = df.copy()
    
    for column in notnan_columns:
        df_copy[column].dropna(inplace=True)
    
    return df_copy


In [58]:
def format_listcolumns(df, column):
    """
    Wandelt eine Spalte mit Listen als Strings formatiert in echte Listen um und gibt ein DataFrame und die eindeutigen Werte zurück.

    Parameters:
    df (pd.DataFrame): Der DataFrame, der die Spalte enthält.
    column (str): Der Name der Spalte, die konvertiert werden soll.

    Returns:
    pd.DataFrame: Das DataFrame mit der umgewandelten Spalte.
    list: Eine Liste der eindeutigen Werte in der umgewandelten Spalte.

    Example:
    >>> df, unique_values = format_listcolumns(df_train, 'features')
    """
    # Kopie der Spalte erstellen, um die Originaldaten nicht zu ändern
    df_copy = df.copy()

    # Umwandlung der Spalte von einem String in eine Liste
    df_copy[column].replace({'\\n': ''}, regex=True, inplace=True)
    df_copy = mytools.df_string_to_list(df_copy, column, entry_delimiter="'", separator=' ')

    # Eindeutige Werte in der umgewandelten Spalte finden
    unique_values = mytools.df_unique_list_values(df_copy, column)

    return df_copy, unique_values

In [59]:
#funtion to reorganize a column of subtopics into  broader topics and removing some of them 
def categorize_topics(df,column,topics,remove):
    df_copy = df.copy()
    
    for topic in topics:
        for subtopic in topic[0]:
            df_copy[column] = df_copy[column].replace({'\'' + subtopic + '\'': '\'' + topic[1] + '\''}, regex=True)
    
    for subtopic in remove:
        df_copy[column] = df_copy[column].replace({'\'' + subtopic + '\'': ''}, regex=True)
    
    df_copy[column] = df_copy[column].replace({' ': ''}, regex=True)
    df_copy[column] = series_to_list(df_copy, column)
    df_copy = df_copy[df_copy[column].str.len() == 1]
    df_copy[column] = df_copy[column].apply(lambda x: x[0])
    
    return df_copy

In [60]:
def format_datecolumns(df,date_columns):
    # Kopie des DataFrame erstellen, um die Originaldaten nicht zu ändern
    df_copy = df.copy()

    for column in date_columns:
        # Die Zeichenkette in ein Datum konvertieren
        df_copy[column] = pd.to_datetime(df_copy[column].str.strip().str.split(' ').str.get(0))
        df_copy[column+'_month'] = df_copy[column].dt.month

        # Woche extrahieren (altes Verhalten, ab Pandas 1.1.0 ist isocalendar().week empfohlen)
        df_copy[column+'_day_month'] = df_copy[column].dt.day

        # Tag extrahieren
        df_copy[column+'_day_year'] = df_copy[column].dt.day_of_year

        # Wochentag extrahieren (Montag=0, Sonntag=6)
        df_copy[column+'_weekday'] = df_copy[column].dt.day_name('en')

        df_copy[column+'_quarter_year'] = df_copy[column].dt.quarter
        df_copy = pd.get_dummies(df_copy, columns=[column+'_weekday'])
        weekdays = ['weekday_Monday', 'weekday_Tuesday', 'weekday_Wednesday', 'weekday_Thursday', 'weekday_Friday', 'weekday_Saturday', 'weekday_Sunday']
        for weekday in weekdays:
            if not column+'_'+weekday in df_copy.columns:
                df_copy[column+'_'+weekday] = 0
            else:
                df_copy[column+'_'+weekday] = df_copy[column+'_'+weekday].astype(int)

        df_copy = df_copy.drop(columns=column)

    return df_copy

In [61]:
def format_textcolumns(df,text_columns):
    # Kopie des DataFrame erstellen, um die Originaldaten nicht zu ändern
    df_copy = df.copy()

    for column in text_columns:
        df_copy[column].replace({'&lt;': '<'}, regex=True, inplace=True)
        df_copy[column].replace({'\\n': ' '}, regex=True, inplace=True)
        df_copy[column] = df_copy[column].str.replace('\s+', ' ', regex=True)
        df_copy[column] = df_copy[column].str.lower()
        df_copy[column] = df_copy[column].fillna(value='')

    return df_copy

<>:8: SyntaxWarning: invalid escape sequence '\s'
<>:8: SyntaxWarning: invalid escape sequence '\s'
C:\Users\tilll\AppData\Local\Temp\ipykernel_30244\1029708371.py:8: SyntaxWarning: invalid escape sequence '\s'
  df_copy[column] = df_copy[column].str.replace('\s+', ' ', regex=True)


In [62]:
def handle_special_columns(df,list_column,list_possible_values,drop_columns,date_columns,notnan_columns,text_columns):
    # Kopie des DataFrame erstellen, um die Originaldaten nicht zu ändern
    df_copy = df.copy()

    # Spalten aus dem DataFrame entfernen
    df_copy = df_copy.drop(columns=drop_columns)

    # Spalte mit Listen explodieren und mögliche Werte festlegen
    df_copy = mytools.df_explode_listcolumn(df_copy, list_column, list_possible_values)

    # Datumsangaben formatieren
    df_copy = format_datecolumns(df_copy, date_columns)

    # Zeilen entfernen, die NaN-Werte in bestimmten Spalten enthalten
    df_copy = drop_row_notnan_columms(df_copy, notnan_columns)

    # Textspalten formatieren
    df_copy = format_textcolumns(df_copy, text_columns)

    return df_copy

In [63]:
#has to expanded to make it readeble by model
def preprocessing(df,topic_column,topics,topics_to_remove,list_column,list_possible_values,drop_columns,date_columns,notnan_columns,text_columns,encoder,fit_encoder=True):
    df_copy = df.copy()
    
    df_copy = categorize_topics(df_copy, topic_column, topics, topics_to_remove)
    df_copy = handle_special_columns(df_copy, list_column, list_possible_values, drop_columns, date_columns, notnan_columns, text_columns)
    
    additional_features = df_copy.drop(columns=(text_columns + [topic_column]))
    
    if fit_encoder:
        labels = torch.tensor(encoder.fit_transform(df_copy[topic_column]))
    else:
        labels = torch.tensor(encoder.transform(df_copy[topic_column]))
    labels = labels.long()
    
    return df_copy[text_columns], additional_features, labels, encoder

#### Actual Preprocessing

In [64]:
df,unique_values_test = format_listcolumns(df_test,list_column)
df,unique_values_train = format_listcolumns(df_train,list_column)
unique_countries=unique_values_train

C:\Users\tilll\AppData\Local\Temp\ipykernel_30244\2357064120.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy[column].replace({'\\n': ''}, regex=True, inplace=True)


In [65]:
if load_encoder:
    label_encoder = joblib.load('label_encoder.joblib')
else:
   label_encoder = LabelEncoder() 

In [66]:

train_df_text,train_additional_features,train_labels,label_encoder = preprocessing(df_train,topic_column,topics,topics_to_remove,list_column,unique_countries,drop_columns,date_columns,notnan_columns,text_columns,label_encoder,fit_encoder=fit_encoder)

test_df_text,test_additional_features,test_labels,label_encoder = preprocessing(df_test,topic_column,topics,topics_to_remove,list_column,unique_countries,drop_columns,date_columns,notnan_columns,text_columns,label_encoder,fit_encoder=fit_encoder)

if not load_encoder:
    joblib.dump(label_encoder, 'label_encoder.joblib')

C:\Users\tilll\AppData\Local\Temp\ipykernel_30244\3411280864.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_copy[column] = pd.to_datetime(df_copy[column].str.strip().str.split(' ').str.get(0))
C:\Users\tilll\AppData\Local\Temp\ipykernel_30244\1029708371.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy[column].replace({'&lt;': '<'}, regex=True, inplace=True)
C:\Users\till

In [67]:
# Z-Score-Encoder erstellen
fit_encoder = False
zscore_encoder = mytools.ZScoreEncoder()
standardize_columns = ['date_month', 'date_day_month', 'date_day_year', 'date_quarter_year']

if fit_encoder:
    train_additional_features = zscore_encoder.fit_transform(train_additional_features, standardize_columns)
else:
    zscore_encoder.load('zscore_encoder.joblib')
    train_additional_features = zscore_encoder.transform(train_additional_features)

test_additional_features = zscore_encoder.transform(test_additional_features)

# Z-Score-Encoder speichern
if fit_encoder:
    zscore_encoder.save('zscore_encoder.joblib')

#convert additional features to tensor
train_additional_features = torch.tensor(train_additional_features.values).float()
test_additional_features = torch.tensor(test_additional_features.values).float()



# Erstellung des Modell

#### Definieren der Parameter

In [68]:
model_name='bert-base-multilingual-uncased'
num_additional_features=139
num_classes=5
freeze_bert=True
num_epochs=5
batch_size=32
model_path_base='models/Bert_freeze_extended'
train_epochs=5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


#### Initilisieren vom Tokennizer

In [69]:
tokenizer = BertTokenizer.from_pretrained(model_name)

## Definieren der benötigten Funktionen und Objekte

#### Tokenizen und Dataset

In [70]:
def tokenize_texts(text,length=128):
    tokenized_text = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=length)
    return tokenized_text

def tokenize_inputs(df_text):
    tokenized_inputs1 = []
    tokenized_inputs2 = []
    for idx, row in df_text.iterrows():
        inputs1 = tokenize_texts(row['text'],256)
        inputs2 = tokenize_texts(row['title'],16)
        tokenized_inputs1.append(inputs1)
        tokenized_inputs2.append(inputs2)
    return tokenized_inputs1,tokenized_inputs2

In [71]:
class Text_Text_Feature_Dataset(torch.utils.data.Dataset):
    def __init__(self, tokenized_inputs1, tokenized_inputs2, additional_features, labels):
        self.tokenized_inputs1 = tokenized_inputs1
        self.tokenized_inputs2 = tokenized_inputs2
        self.additional_features = additional_features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input_ids1 = self.tokenized_inputs1[idx]['input_ids'].squeeze()
        attention_mask1 = self.tokenized_inputs1[idx]['attention_mask'].squeeze()
        input_ids2 = self.tokenized_inputs2[idx]['input_ids'].squeeze()
        attention_mask2 = self.tokenized_inputs2[idx]['attention_mask'].squeeze()
        additional_features = self.additional_features[idx]
        label = self.labels[idx]
        return input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features, label

#### Modell

In [72]:
class MultilingualBERTClassifier(nn.Module):
    def __init__(self, bert_model_name='bert-base-multilingual-uncased', num_additional_features=119, num_classes=5, freeze_bert=True,hidden_layers=None):
        super(MultilingualBERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        
        # Einfrieren der BERT-Gewichte, falls angegeben
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
        self.additional_features_layers = nn.Sequential(
            nn.Linear(num_additional_features, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.dropout = nn.Dropout(0.1)
        
        # Anzahl der Neuronen im Input-Layer des Klassifikators (BERT Output * 2 + zusätzliche Features)
        input_size = 768 * 2 + 128

        # Erstellen der Hidden Layers
        if hidden_layers is None:
            hidden_layers = []
        layers = []
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.1))
            input_size = hidden_size

        # Finaler Klassifikations-Layer
        layers.append(nn.Linear(input_size, num_classes))
        self.classifier = nn.Sequential(*layers)
        
        # Speichern der Initialisierungsparameter
        self.init_params = {
            'bert_model_name': bert_model_name,
            'num_additional_features': num_additional_features,
            'num_classes': num_classes,
            'freeze_bert': freeze_bert,
            'hidden_layers': hidden_layers
        }

    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features):
        outputs1 = self.bert(input_ids1, attention_mask=attention_mask1)
        pooled_output1 = outputs1[1]  # [CLS] token representation
        
        outputs2 = self.bert(input_ids2, attention_mask=attention_mask2)
        pooled_output2 = outputs2[1]  # [CLS] token representation
        
        additional_features_output = self.additional_features_layers(additional_features)
        
        combined_output = torch.cat((pooled_output1, pooled_output2, additional_features_output), dim=1)
        combined_output = self.dropout(combined_output)
        
        logits = self.classifier(combined_output)
        
        return logits

#### Evaluation

In [73]:
def calculate_and_append_metrics(epoch, classes, labels, predictions,df=None):
    # Confusion Matrix erstellen 
    cm = confusion_matrix(labels, predictions, labels=classes)
    # Spalten für das DataFrame definieren
    columns = ['Epoch', 'Class', 'TP abs', 'True Positive', 'TP %', 'FP abs', 'False Positive', 'FP %', 'FN abs','False Negative', 'FN %']
    data = []

    # Iteration über die Klassen
    for idx, class_name in enumerate(classes):
        total = sum(1 for label in labels if label == class_name)
        true_positive = cm[idx, idx]
        false_positive = cm[:, idx].sum() - true_positive
        false_negative = cm[idx,:].sum() - true_positive

        tp_percent = (true_positive / total) * 100 if total > 0 else 0
        fp_percent = (false_positive / total) * 100 if total > 0 else 0
        fn_percent = (false_negative / total) * 100 if total > 0 else 0

        tp_text = f"{round(tp_percent,2)}% ({true_positive})"
        fp_text = f"{round(fp_percent,2)}% ({false_positive})"
        fn_text = f"{round(fn_percent,2)}% ({false_negative})"


        data.append([epoch, class_name, true_positive, tp_text, tp_percent, false_positive, fp_text, fp_percent, false_negative, fn_text, fn_percent])

    # DataFrame für die aktuelle Epoche erstellen
    epoch_df = pd.DataFrame(data, columns=columns)
    epoch_df.set_index(['Epoch', 'Class'],inplace=True)
    if df is None:
        return epoch_df
    else:
        return pd.concat([df, epoch_df])

In [74]:
def train(model, dataloader, criterion, optimizer, scheduler, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for batch in tqdm(dataloader, desc=f'Epoch {epoch + 1}'):
            input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features, labels = batch
            input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
            input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)
            additional_features, labels = additional_features.to(device), labels.to(device)

            optimizer.zero_grad()
            
            # Vorwärtsdurchlauf
            logits = model(input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features)

            # Verlust berechnen
            loss = criterion(logits, labels)
            epoch_loss += loss.item()
            
            # Rückwärtsdurchlauf und Optimierung
            loss.backward()
            optimizer.step()
        
        avg_loss = epoch_loss / len(dataloader)
        scheduler.step(avg_loss)
        
        print(f"Epoch {epoch + 1} (BERT eingefroren), Loss: {avg_loss}")

In [75]:
def eval(model,counter,test_dataset,df=None):    
    #test modell
    model.eval()
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
    test_predictions = []
    test_labels = []

    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features, labels = batch
            input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
            input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)
            additional_features, labels = additional_features.to(device), labels.to(device)

            logits = model(input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features)
            predictions = torch.argmax(logits, dim=1)
            
            test_predictions.extend(predictions.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    test_predictions = label_encoder.inverse_transform(test_predictions)
    test_labels = label_encoder.inverse_transform(test_labels)

    accuracy = sum(test_predictions == test_labels) / len(test_labels)
    print(f'Accuracy: {accuracy}')

    classes=[topic[1] for topic in topics] 
    if counter==1:
        df=calculate_and_append_metrics(counter, classes, test_labels, test_predictions)
    else:
        df=calculate_and_append_metrics(counter, classes, test_labels, test_predictions,df)
    counter+=1

    
    return df,counter

## Training

#### Erstellen des Datasets

In [76]:
#Tokenize Text
train_tokenized_inputs1, train_tokenized_inputs2= tokenize_inputs(train_df_text)
test_tokenized_inputs1, test_tokenized_inputs2= tokenize_inputs(test_df_text)
#Create Datasets
train_dataset = Text_Text_Feature_Dataset(train_tokenized_inputs1, train_tokenized_inputs2,train_additional_features,train_labels)
test_dataset = Text_Text_Feature_Dataset(test_tokenized_inputs1, test_tokenized_inputs2,test_additional_features,test_labels)

#### Initialisieren von Modell

In [79]:
# Create or load model
model = mytools.modelversions_load_model('versions',model_path_base=model_path_base)
if model is None:
    # If no model was loaded, initialize a new one
    model = MultilingualBERTClassifier(num_additional_features=num_additional_features, num_classes=num_classes,hidden_layers=[256, 128])
    print('New model created.')
else:
    print('Model loaded from saved version.')

# Move model to device if possible
model.to(device)
print('Model on device')

Please enter a valid mode


AttributeError: 'NoneType' object has no attribute 'seek'. You can only torch.load from a file that is seekable. Please pre-load the data into a buffer like io.BytesIO and try to load from it instead.

In [40]:
#create Dataloader
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# Loss-Funktion and Optimizer
criterion = nn.CrossEntropyLoss(weight=mytools.train_calculate_class_weights(train_labels).to(device))
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)

C:\Users\tilll\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


#### Training

In [48]:
counter=1

In [34]:
for i in range(train_epochs):
    if i>0:
        metrics_df = df.loc[df.index.get_level_values('Epoch') >= (df.index.get_level_values('Epoch').max() - 2)] 
        columns = pd.MultiIndex.from_product([metrics_df.columns.get_level_values(0).unique(), ['True Positive', 'False Positive', 'False Negative']]) 
        metrics_table = pd.DataFrame(index=metrics_df.index.levels[0], columns=columns) 
        metrics_df = metrics_df[['True Positive','False Positive','False Negative']] 
        metrics_df = metrics_df.stack().unstack(1)
        print(metrics_df)
    train(model, dataloader, criterion, optimizer, scheduler, num_epochs)
    if i==0:
        df,counter=eval(model,counter)
    else:
        df,counter=eval(model,counter,df)
metrics_df

Epoch 1:   0%|          | 0/199 [00:00<?, ?it/s]C:\Users\tilll\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Epoch 1: 100%|██████████| 199/199 [01:04<00:00,  3.07it/s]


Epoch 1 (BERT eingefroren), Loss: 0.6543200727383695


Epoch 2: 100%|██████████| 199/199 [01:02<00:00,  3.20it/s]


Epoch 2 (BERT eingefroren), Loss: 0.6505842026154599


Epoch 3: 100%|██████████| 199/199 [01:02<00:00,  3.21it/s]


...

Epoch 5: 100%|██████████| 199/199 [01:00<00:00,  3.29it/s]


Epoch 5 (BERT eingefroren), Loss: 0.6312347738886598


100%|██████████| 79/79 [00:23<00:00,  3.41it/s]

Accuracy: 0.8687150837988827


Class                      finance         food personal_finance  \
Epoch                                                              
7     True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   
8     True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   
9     True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   

Class                     resource    transport  
Epoch                                            
7     True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)  
8     True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)  
9     True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)

In [35]:
metrics_df = df.loc[df.index.get_level_values('Epoch') >= (df.index.get_level_values('Epoch').max() - 2)] 
columns = pd.MultiIndex.from_product([metrics_df.columns.get_level_values(0).unique(), ['True Positive', 'False Positive', 'False Negative']]) 
metrics_table = pd.DataFrame(index=metrics_df.index.levels[0], columns=columns) 
metrics_df = metrics_df[['True Positive','False Positive','False Negative']] 
metrics_df.stack().unstack(1)

Class                      finance         food personal_finance  \
Epoch                                                              
8     True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   
9     True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   
10    True Positive   95.61% (936)  76.38% (97)     88.71% (974)   
      False Positive  22.06% (216)  17.32% (22)       1.09% (12)   
      False Negative    4.39% (43)  23.62% (30)     11.29% (124)   

Class                     resource    transport  
Epoch                                            
8     True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)  
9     True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)  
10    True Positive   58.87% (156)  37.84% (14)  
      False Positive   26.04% (69)  27.03% (10)  
      False Negative  41.13% (109)  62.16% (23)

In [49]:
import torch.nn.functional as F

# Modell in den Evaluationsmodus setzen
model.eval()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
probabilities = torch.empty(0, num_classes)
predictions = torch.empty(0)
max_probabilities = torch.empty(0)

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features, labels = batch
        input_ids1, attention_mask1 = input_ids1.to(device), attention_mask1.to(device)
        input_ids2, attention_mask2 = input_ids2.to(device), attention_mask2.to(device)
        additional_features, labels = additional_features.to(device), labels.to(device)

        batch_logits = model(input_ids1, attention_mask1, input_ids2, attention_mask2, additional_features)
        #shift to cpu
        batch_probabilities = F.softmax(batch_logits, dim=1).cpu()
        batch_predictions = torch.argmax(batch_logits, dim=1).cpu()
        batch_max_probabilities = torch.max(batch_probabilities, dim=1).values

        probabilities = torch.cat((probabilities, batch_probabilities), dim=0)
        predictions = torch.cat((predictions, batch_predictions), dim=0)
        max_probabilities = torch.cat((max_probabilities, batch_max_probabilities), dim=0)

# Schwellenwert definieren
threshold = 0.7

# Sichere und unsichere Vorhersagen unterscheiden
secure_predictions = max_probabilities >= threshold
insecure_predictions = max_probabilities < threshold

# Ausgabe der sicheren und unsicheren Vorhersagen
secure_results = predictions[secure_predictions]
insecure_results = predictions[insecure_predictions]

# True Labels für beide Gruppen
secure_labels = test_labels[secure_predictions]
insecure_labels = test_labels[insecure_predictions]

def calculate_accuracy(predictions, labels):
    correct = (predictions == labels).sum().item()
    total = labels.size(0)
    return correct / total

# Accuracy für sichere Vorhersagen
if secure_labels.size(0) > 0:
    secure_accuracy = calculate_accuracy(secure_results, secure_labels)
else:
    secure_accuracy = 0  # Keine sicheren Vorhersagen

# Accuracy für unsichere Vorhersagen
if insecure_labels.size(0) > 0:
    insecure_accuracy = calculate_accuracy(insecure_results, insecure_labels)
else:
    insecure_accuracy = 0  # Keine unsicheren Vorhersagen

# Ausgabe der Accuracy für beide Gruppen
print(f'Sichere Vorhersagen Accuracy: {secure_accuracy:.4f}')
print(f'Unsichere Vorhersagen Accuracy: {insecure_accuracy:.4f}')

#anzahl der sicheren und unsicheren vorhersagen
num_secure = secure_labels.size(0)
num_insecure = insecure_labels.size(0)

# Ausgabe der Anzahl der sicheren und unsicheren Vorhersagen
print(f'Anzahl sichere Vorhersagen: {num_secure}')
print(f'Anzahl unsichere Vorhersagen: {num_insecure}')

100%|██████████| 79/79 [00:23<00:00,  3.30it/s]

Sichere Vorhersagen Accuracy: 0.9617
Unsichere Vorhersagen Accuracy: 0.5908
Anzahl sichere Vorhersagen: 1878
Anzahl unsichere Vorhersagen: 628


#### Saving Modell

In [36]:
mytools.modelversions_save_model(model, model_path_base)

Model saved as models/Bert_freeze_extended_v2.pt
